
# Pandas
## More binning;  Melting; Pivoting



In [ ]:
import pandas as pd
import numpy as np

np.random.seed(1)

In [44]:
arr = np.random.multivariate_normal(mean=[1,0.5], cov=[[1,0],[0,1]], size=10000)
arr.shape

(10000, 2)

In [45]:
df = pd.DataFrame(arr, columns=['x1','x2'])

In [4]:
df.head()

,x1,x2
0,2.624345,-0.111756
1,0.471828,-0.572969
2,1.865408,-1.801539
3,2.744812,-0.261207
4,1.319039,0.250630


In [7]:
# one type of binning (we saw last week)
pd.cut(df['x1'], bins=[-np.inf, -1, 0, 1, np.inf], labels=[1,2,3,4])

0       4
1       3
2       4
3       4
4       4
       ..
9995    4
9996    1
9997    4
9998    4
9999    4
Name: x1, Length: 10000, dtype: category
Categories (4, int64): [1 < 2 < 3 < 4]

In [46]:
# another type of binning using qcut (quartiles)
df['quartile_x1'] = pd.qcut(df['x1'], q=[0, .25, .5, .75, 1], labels=['1st Qtile', '2nd Qtile', '3rd Qtile', '4th Qtile'])

In [12]:
df.head()

,x1,x2,quartile_x1
0,2.624345,-0.111756,4th Qtile
1,0.471828,-0.572969,2nd Qtile
2,1.865408,-1.801539,4th Qtile
3,2.744812,-0.261207,4th Qtile
4,1.319039,0.250630,3rd Qtile


In [16]:
df.groupby(['quartile_x1'])['x2'].agg(['mean','std']).reset_index()

,quartile_x1,mean,std
0,1st Qtile,0.530831,1.005354
1,2nd Qtile,0.474668,0.984340
2,3rd Qtile,0.518893,1.006167
3,4th Qtile,0.481867,0.981813


In [19]:
df.groupby(['quartile_x1']).agg({'x1':'mean','x2':['std','var']})

x1        x2          
                 mean       std       var
quartile_x1                              
1st Qtile   -0.263848  1.005354  1.010736
2nd Qtile    0.693346  0.984340  0.968924
3rd Qtile    1.351169  1.006167  1.012371
4th Qtile    2.289905  0.981813  0.963957

In [47]:
# this just pulls that gruop mean value into the df as a column; 
#   note it repeats the same value per quartile
df['mean_x2_by_qx1'] = df.groupby(['quartile_x1'])['x2'].transform('mean')

In [22]:
df

,x1,x2,quartile_x1,mean_x2_by_qx1
0,2.624345,-0.111756,4th Qtile,0.481867
1,0.471828,-0.572969,2nd Qtile,0.474668
2,1.865408,-1.801539,4th Qtile,0.481867
3,2.744812,-0.261207,4th Qtile,0.481867
4,1.319039,0.250630,3rd Qtile,0.518893
...,...,...,...,...
9995,1.383964,-0.318778,3rd Qtile,0.518893
9996,-1.124622,-0.921937,1st Qtile,0.530831
9997,2.109570,-0.443208,4th Qtile,0.481867
9998,1.782216,2.908434,4th Qtile,0.481867


In [48]:
#want to compute x2 / sum(x2)  within each bucket
df['sum_x2_by_qx1'] = df.groupby(['quartile_x1'])['x2'].transform('sum')


In [49]:
df['ratio_x2_to_sumx2'] = df['x2']/df['sum_x2_by_qx1']

In [50]:
# again: want to compute x2 / sum(x2)  within each bucket
udf_ratio = lambda x: x/x.sum()

# same as defining as a fucntion



In [51]:
df['ratio_x2_to_sumx2'] = df.groupby(['quartile_x1'])['x2'].apply(udf_ratio)

In [28]:
df

,x1,x2,quartile_x1,mean_x2_by_qx1,sum_x2_by_qx1,ratio_x2_to_sumx2
0,2.624345,-0.111756,4th Qtile,0.481867,1204.667016,-0.000093
1,0.471828,-0.572969,2nd Qtile,0.474668,1186.669886,-0.000483
2,1.865408,-1.801539,4th Qtile,0.481867,1204.667016,-0.001495
3,2.744812,-0.261207,4th Qtile,0.481867,1204.667016,-0.000217
4,1.319039,0.250630,3rd Qtile,0.518893,1297.233311,0.000193
...,...,...,...,...,...,...
9995,1.383964,-0.318778,3rd Qtile,0.518893,1297.233311,-0.000246
9996,-1.124622,-0.921937,1st Qtile,0.530831,1327.078695,-0.000695
9997,2.109570,-0.443208,4th Qtile,0.481867,1204.667016,-0.000368
9998,1.782216,2.908434,4th Qtile,0.481867,1204.667016,0.002414


In [52]:
consumption = {'family':[1,1,2,2], 'gender':[0,1,1,0], 'spend':[50,100,75,80]}

In [53]:
df_spend=pd.DataFrame(consumption)
df_spend

,family,gender,spend
0,1,0,50
1,1,1,100
2,2,1,75
3,2,0,80


In [32]:
# calculate each person's portion of the family spending
df_spend['ratio'] = df_spend.groupby('family')['spend'].apply(udf_ratio)
df_spend

,family,gender,spend,ratio
0,1,0,50,0.333333
1,1,1,100,0.666667
2,2,1,75,0.483871
3,2,0,80,0.516129


# Melt and Pivot

Perform opposite / complementary functions:


*   melt transforms from wide to long
*  pivot transforms from long to wide



In [82]:
# Let's pare the table back down to just 2 columns
df = df[['x1','x2']]

### Widen table

...by adding a couple columns

In [115]:
# Following gives a 'SettingWithCopyWarning' warning:
#    "A value is trying to be set on a copy of a slice...."
# This is just a warning - could be an issue, depends on the situation
#   warning can be disabled via
#   pd.options.mode.chained_assignment = None  # default='warn'
#   (execute just after the pandas import)

df['x3'] = df.loc[:, 'x1'] + df.loc[:, 'x2']
df['x4'] = df.loc[:,'x1'] * df.loc[:,'x2']

In [90]:
df

,x1,x2,x3,x4
0,2.624345,-0.111756,2.512589,-0.293287
1,0.471828,-0.572969,-0.101140,-0.270343
2,1.865408,-1.801539,0.063869,-3.360604
3,2.744812,-0.261207,2.483605,-0.716964
4,1.319039,0.250630,1.569669,0.330590
...,...,...,...,...
9995,1.383964,-0.318778,1.065186,-0.441178
9996,-1.124622,-0.921937,-2.046559,1.036830
9997,2.109570,-0.443208,1.666362,-0.934979
9998,1.782216,2.908434,4.690650,5.183457


In [103]:
# Here we go - melt from wide to long table - note this only uses cols x1 and x2
#df_melted = pd.melt(df, value_vars=['x1','x2'], value_name='x')
#...this version uses all (non-id) columns
df_melted = pd.melt(df, value_name='x')

In [104]:
df_melted

,variable,x
0,x1,2.624345
1,x1,0.471828
2,x1,1.865408
3,x1,2.744812
4,x1,1.319039
...,...,...
39995,x4,-0.441178
39996,x4,1.036830
39997,x4,-0.934979
39998,x4,5.183457


In [105]:
# let's see what we have
df_melted.groupby(['variable']).size()

variable
x1    10000
x2    10000
x3    10000
x4    10000
dtype: int64

### Pivot back to original layout

We melted wide to long.....now revert back to wide.

In [108]:
# Change back to original layout

# this is how you would do it - currently there is an issue ****
#pd.pivot_table(df_melted,columns='variable')
#pd.pivot(df_melted,values='x',columns='variable')
# ** still broken ***

RangeIndex(start=0, stop=40000, step=1)

## Mimicing melting 

Splitting df into smaller df's that need to be combined.

I.e. these are "wide" because they are in multiple/parallel dfs.

Often you don't care where the data came from you're just merging streams.

### Example
Pipeline workflow::
```
output -> shard1.csv, shard2.csv... df1, df2, df3, ... 
pd.concat([df1, df2, df3...])
```



In [116]:
# Create a couple parallel dfs
df_x1 = df[['x1']]
df_x2 = df[['x2']]

# (optional) can add labels to show source of each data point
df_x1['label'] = 'x1'
df_x2['label'] = 'x2'

In [117]:
df_x1.head()


,x1,label
0,2.624345,x1
1,0.471828,x1
2,1.865408,x1
3,2.744812,x1
4,1.319039,x1


In [118]:
# start by renaming columns so concatenation does not require column mapping
df_x1.rename(columns={'x1':'x'}, inplace=True)
df_x2.rename(columns={'x2':'x'}, inplace=True)

In [119]:
df_x2.head()

,x,label
0,-0.111756,x2
1,-0.572969,x2
2,-1.801539,x2
3,-0.261207,x2
4,0.250630,x2


In [120]:
df_stacked = pd.concat([df_x1, df_x2])

In [59]:
# note, like the melted when we ended up with 40K rows
df_stacked

,x
0,2.624345
1,0.471828
2,1.865408
3,2.744812
4,1.319039
...,...
9995,-0.318778
9996,-0.921937
9997,-0.443208
9998,2.908434


### Combining via Joins

Takes similar datasets and makes a single wider df

In [62]:
# Join horizontally, not vertically
#    have to differentiate col names - so give suffixes based on left/right
#  This will join on index (could provide a key column to match on)
df_x1.join(df_x2, lsuffix='_l', rsuffix='_r')

,x_l,x_r
0,2.624345,-0.111756
1,0.471828,-0.572969
2,1.865408,-1.801539
3,2.744812,-0.261207
4,1.319039,0.250630
...,...,...
9995,1.383964,-0.318778
9996,-1.124622,-0.921937
9997,2.109570,-0.443208
9998,1.782216,2.908434


In [63]:
# Merge is similar to join but can work on columns, also
#    a more general function
#  this is again using the index from both to join on
df_x1.merge(df_x2, left_index=True, right_index=True)

,x_x,x_y
0,2.624345,-0.111756
1,0.471828,-0.572969
2,1.865408,-1.801539
3,2.744812,-0.261207
4,1.319039,0.250630
...,...,...
9995,1.383964,-0.318778
9996,-1.124622,-0.921937
9997,2.109570,-0.443208
9998,1.782216,2.908434


### ...or sometimes you want to merge on a particular column - can do SQL-style
```
df_x1.merge(df_x2, on=['**colName**'], how='inner'|'left'|'right'|'outer')
```

In [68]:
# example building on the df_spend case
df_spend

,family,gender,spend
0,1,0,50
1,1,1,100
2,2,1,75
3,2,0,80


In [66]:
address={'address':['MD','VA','DC'], 'family':[1,2,3]}

In [69]:
df_address = pd.DataFrame(address)
df_address

,address,family
0,MD,1
1,VA,2
2,DC,3


In [70]:
# inner...keep only matches
df_spend.merge(df_address, on='family', how='inner')

,family,gender,spend,address
0,1,0,50,MD
1,1,1,100,MD
2,2,1,75,VA
3,2,0,80,VA


In [71]:
# outer...keep all
df_spend.merge(df_address, on='family', how='outer')

,family,gender,spend,address
0,1,0.0,50.0,MD
1,1,1.0,100.0,MD
2,2,1.0,75.0,VA
3,2,0.0,80.0,VA
4,3,NaN,NaN,DC


### Splitting data

Use case:  train/test split

Need a training set, and test set
Same data cols, diff rows

Want random selection

In [72]:
# can shuffle the data
index = np.arange(0, len(df))

In [75]:
np.random.seed(1)
np.random.shuffle(index)


In [78]:
# didn't want to modify the original set so making a copy
df_shuffled = df.iloc[index,:].copy()

In [79]:
df_shuffled

,x1,x2,quartile_x1,mean_x2_by_qx1,sum_x2_by_qx1,ratio_x2_to_sumx2
1300,1.256052,-0.215232,3rd Qtile,0.518893,1297.233311,-0.000166
7455,0.973444,1.412668,2nd Qtile,0.474668,1186.669886,0.001190
9610,-0.770253,-1.388160,1st Qtile,0.530831,1327.078695,-0.001046
6491,0.190624,0.556986,1st Qtile,0.530831,1327.078695,0.000420
7054,1.456499,0.801692,3rd Qtile,0.518893,1297.233311,0.000618
...,...,...,...,...,...,...
9400,1.952001,1.187254,4th Qtile,0.481867,1204.667016,0.000986
79,1.017409,-0.622019,2nd Qtile,0.474668,1186.669886,-0.000524
1668,0.536816,-0.830769,2nd Qtile,0.474668,1186.669886,-0.000700
8353,0.907732,0.325525,2nd Qtile,0.474668,1186.669886,0.000274


In [ ]:
# Of course you could always use a function vs. a lambda  (is there any perf diff?)
#def splitting(df, train_percent):
# ... you finish here